In [62]:
#必要なライブラリとモジュールのインポート
import os
import sys
import json
import ccxt
import pandas as pd
import numpy as np
import talib
from datetime import datetime, timedelta

In [63]:


# APIキーの読み込み
with open("api_key.json", "r") as f:
    api_keys = json.load(f)

# 環境設定
TEST_MODE = False  # True: テスト環境, False: 本番環境
PIPS = 0.001  # 価格の最小単位（例: pips）

# CCXTでのAPI設定
exchange = ccxt.hyperliquid({
    "apiKey": api_keys["apiKey"],
    "privateKey": api_keys["privateKey"],
    "walletAddress": api_keys["walletaddress"],
    "timeout": 20000  # タイムアウトを20秒に設定
})

# ウォレットアドレスの取得
walletAddress = api_keys['walletaddress']

exchange.verbose = False  # デバッグ情報を非表示にする

# 標準出力を一時的に無効化
class SuppressOutput:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# CCXTの情報出力を抑制
with SuppressOutput():
    exchange.load_markets()



In [64]:
#データフレームの作成
# 通貨ペアと期間
symbol = "HYPE/USDC:USDC"
timeframe = "1m"  # 時間足（1分足）
atr_period = 14  # ATRの期間

# OHLCデータ取得
def fetch_ohlc_data(symbol, timeframe, since):
    ohlc = exchange.fetch_ohlcv(symbol, timeframe, since=since)
    df = pd.DataFrame(ohlc, columns=["timestamp", "open", "high", "low", "close", "volume"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    return df

# ATRの計算
def calculate_atr(df, period):
    df["ATR"] = talib.ATR(df["high"], df["low"], df["close"], timeperiod=period)
    return df

# 指値価格の計算
def calculate_limit_price(df, pips):
    df["limit_price_dist"] = df["ATR"] * 0.5  # ハイパーパラメータ
    df["limit_price_dist"] = np.maximum(1, (df["limit_price_dist"] / pips).round().fillna(1)) * pips
    return df

# レバレッジ設定
#def set_leverage(symbol, leverage): #hyperliquidのccxtはset_leverageをサポートしていない。
    #try:
        #response = exchange.private_post_set_leverage({
           # "symbol": symbol,
           # "leverage": leverage
       # })
       # print(f"Leverage set to {leverage}x for {symbol}: {response}")
    #except Exception as e:
        #print(f"Failed to set leverage: {e}")

# USDCの残高を取得
def get_usdc_balance():
    try:
        balance = exchange.fetch_balance(params={"user": api_keys["walletaddress"]})
        usdc_balance = balance.get("USDC", {}).get("free", 0)  # USDCのフリーバランスを取得
        return usdc_balance
    except Exception as e:
        print(f"Failed to fetch USDC balance: {e}")
        return 0

# 証拠金残高を確認 (USDCのみ)
def check_margin(symbol, amount, price, leverage):
    try:
        if price is None or price <= 0:
            print("Invalid price for margin check.")
            return False
        required_margin = (amount * float(price)) / leverage
        usdc_balance = get_usdc_balance()  # get_usdc_balanceを再利用
        if usdc_balance < required_margin:
            print(f"Insufficient margin: Required {required_margin}, Available {usdc_balance}")
            return False
        return True
    except Exception as e:
        print(f"Failed to fetch margin information: {e}")
        return False

In [65]:
def calculate_dynamic_order_amount(walletAddress, leverage, price):
    """
    動的な注文量を計算 (USDCベース -> HYPE単位に変換)
    - 指定された価格（price）でUSDCをHYPE単位に変換
    """
    usdc_balance = get_usdc_balance()
    if usdc_balance <= 0:
        print("Insufficient balance.")
        return 0.0

    # 動的注文量をUSDCで計算
    dynamic_amount_usdc = usdc_balance * 3 * 0.05 #レバレッジ設定ができる場合は"usdc_balance * leverage * 0.05"

    # HYPE単位に変換
    amount_in_hype = dynamic_amount_usdc / price  # USDCを価格で割りHYPE単位に
    symbol_info = exchange.market(symbol)
    min_amount = symbol_info['limits']['amount']['min'] or 0.5  # 最小取引量

    # 最小単位を考慮して丸める
    amount_in_hype = max(min_amount, round(amount_in_hype, 6))

    print(f"Dynamic order amount (HYPE): {amount_in_hype} (Based on USDC: {dynamic_amount_usdc} and Price: {price})")
    return amount_in_hype

def place_orders_based_on_strategy(df, symbol, walletAddress, leverage):
    """
    売買戦略に基づいて指値注文を送信
    - 指値金額を利用して注文量を計算
    """
    last_row = df.iloc[-1]
    buy_price = last_row['buy_price']
    #sell_price = last_row['sell_price']

    # 買い注文量を計算
    buy_amount = calculate_dynamic_order_amount(walletAddress, leverage, buy_price)
    return buy_amount
    # 売り注文量を計算
    #sell_amount = calculate_dynamic_order_amount(walletAddress, leverage, sell_price)

    # 買い注文を送信
    #place_order(symbol, "buy", buy_amount, buy_price, leverage)
    # 売り注文を送信
    #place_order(symbol, "sell", sell_amount, sell_price, leverage)

In [66]:
import time
import numpy as np
from datetime import datetime, timedelta, timezone
from decimal import Decimal

# ATRで指値距離を計算
def calculate_limit_price(df, pips):
    df['limit_price_dist'] = df['ATR'] * 0.5
    df['limit_price_dist'] = np.maximum(1, (df['limit_price_dist'] / pips).round().fillna(1)) * pips
    df['buy_price'] = df['close'] - df['limit_price_dist']

    return df

def get_current_position_amount(symbol):
    """
    現在のポジションサイズを取得
    :param str symbol: 取引ペア
    :return: 現在のポジション量 (float)、存在しない場合は0
    """
    try:
        # 全ポジションを取得
        positions = exchange.fetch_positions()
        print(f"Fetched positions: {positions}")  # デバッグ用

        # シンボルでフィルタリング
        for pos in positions:
            if pos["symbol"] == symbol:  # シンボルが一致
                return pos.get("contracts", 0)  # ポジションサイズを返す
        
        print(f"No positions found for symbol: {symbol}")
    except Exception as e:
        print(f"Failed to fetch current position: {e}")
    return 0


def place_order(symbol, amount, price, leverage):
    """
    新規注文を作成
    :param str symbol: 取引ペア
    :param float amount: 注文量
    :param float price: 注文価格
    :param float leverage: レバレッジ
    """
    if amount <= 0:
        print("Order amount is zero or invalid. Skipping order placement.")
        return

    try:
        balance = exchange.fetch_balance()["free"]["USDC"]
        required_margin = price * amount / leverage
        if balance < required_margin:
            print(f"Insufficient balance for the order. Required: {required_margin}, Available: {balance}")
            return

        if TEST_MODE:
            print(f"[TEST MODE] Order simulated: buy {amount} {symbol} at {price} with leverage {leverage}x")
        else:
            exchange.create_limit_order(symbol, "buy", amount, price)
            print(f"Order placed: buy {amount} {symbol} at {price}")
    except Exception as e:
        print(f"Failed to place order: {e}")

def place_reduce_only_order(symbol, amount, price):
    """
    Reduce-Onlyの指値注文を作成
    :param str symbol: 取引ペア
    :param float amount: 注文量
    :param float price: 指値価格
    """
    if amount <= 0:
        print("Reduce-Only order skipped: no open position.")
        return

    try:
        if TEST_MODE:
            print(f"[TEST MODE] Reduce-Only Order simulated: sell {amount} {symbol} at {price}")
        else:
            exchange.create_limit_order(symbol, "sell", amount, price, params={"reduceOnly": True})
            print(f"Reduce-Only Order placed: sell {amount} {symbol} at {price}")
    except Exception as e:
        print(f"Failed to place reduce-only order: {e}")

def manage_positions(df, symbol):
    """
    現在のポジションを管理し、必要に応じて決済注文を出す
    :param DataFrame df: データフレーム
    :param str symbol: 取引ペア
    """
    position_amount = get_current_position_amount(symbol)
    if position_amount == 0:
        print(f"No open positions for {symbol}.")
        return

    last_row = df.iloc[-1]
    buy_price = last_row["buy_price"]
    close_price = last_row["close"]
    limit_price_dist = last_row["limit_price_dist"]

    if close_price > buy_price:
        tp_price = close_price + limit_price_dist
        place_reduce_only_order(symbol, position_amount, tp_price)
    else:
        print(f"Close price ({close_price}) is not higher than entry price ({buy_price}). No action taken.")

def trading_bot():
    """
    トレーディングボット本体
    """
    since = int((datetime.now(timezone.utc) - timedelta(minutes=atr_period + 1)).timestamp() * 1000)
    df = fetch_ohlc_data(symbol, timeframe, since)
    if df.empty:
        print("No data fetched. Retrying...")
        return None

    df = calculate_limit_price(calculate_atr(df, atr_period), PIPS)

    leverage = 3  # レバレッジ
    buy_price = df.iloc[-1]["buy_price"]
    amount = calculate_dynamic_order_amount(walletAddress, leverage, buy_price)

    # 新規注文
    place_order(symbol, amount, buy_price, leverage)

    # ポジション管理
    manage_positions(df, symbol)

    return df


# テスト用フラグ
SINGLE_RUN = True  # True: 単発実行, False: 無限ループ

if __name__ == "__main__":
    try:
        if SINGLE_RUN:
            df = trading_bot()
            if df is not None:
                print(df)
            print("Single run complete.")
        else:
            while True:
                df = trading_bot()
                if df is not None:
                    print(df)
                print("Waiting for the next cycle...")
                time.sleep(60)
    except KeyboardInterrupt:
        print("Bot stopped manually.")
    except Exception as e:
        print(f"Error running bot: {e}")





Dynamic order amount (HYPE): 0.5 (Based on USDC: 4.0169292 and Price: 24.691)
Order placed: buy 0.5 HYPE/USDC:USDC at 24.691
Fetched positions: [{'info': {'type': 'oneWay', 'position': {'coin': 'HYPE', 'szi': '1.6', 'leverage': {'type': 'cross', 'value': '3'}, 'entryPx': '24.7337', 'positionValue': '39.616', 'unrealizedPnl': '0.04202', 'returnOnEquity': '0.00318543', 'liquidationPx': None, 'marginUsed': '13.205333', 'maxLeverage': '3', 'cumFunding': {'allTime': '0.578088', 'sinceOpen': '0.0', 'sinceChange': '0.0'}}}, 'id': None, 'symbol': 'HYPE/USDC:USDC', 'timestamp': None, 'datetime': None, 'isolated': False, 'hedged': None, 'side': 'long', 'contracts': 1.6, 'contractSize': 1.0, 'entryPrice': 24.7337, 'markPrice': None, 'notional': 39.616, 'leverage': 3.0, 'collateral': 13.205333, 'initialMargin': 13.205333, 'maintenanceMargin': None, 'initialMarginPercentage': None, 'maintenanceMarginPercentage': None, 'unrealizedPnl': 0.04202, 'liquidationPrice': None, 'marginMode': 'cross', 'perce